<a href="https://colab.research.google.com/github/ryanleeallred/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

The French mathematician Emile Borel once mused that [**infinite monkeys typing for an infinite amount of time**](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type the complete works of William Shakespeare. Let's see if we can get there a bit faster, with the power of **Recurrent Neural Networks and LSTMs**!

Our goal in this projectis to build a **Shakespeare Sonnet Generator**.<br>
Given a prompt of a few words as input, its task is to generate follow-on text that reads like a Shakespeare Sonnet!<br>

To build our Sonnet Generator we will use a type of model called a **sequence model**. Given a short sequence, a sequence  model predicts the **most likely next item in the sequence**. Sequence models are astonishingly versatile and powerful, because the **sequence** we want to predict can be quite general! It can be composed of **words**, or of **characters**, or of **musical notes**, or of data points in a **time series** such as EKG voltages, or stock prices, or even a sequence of **DNA nucleotides**! 

We will train our model on the entire corpus of Shakespeare's Sonnets, and the model will learn from that data the most likely patterns of characters.

In [53]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.callbacks import LambdaCallback

# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Embedding, Bidirectional
# from tensorflow.keras.layers import LSTM

%matplotlib inline

# import a custom text data preparation class

# !wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
from data_cleaning_toolkit_class import data_cleaning_toolkit

### Use `request` to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) to learn how to download the Shakespeare Sonnets from the Gutenberg website. 


In [5]:
# download all of Shakespeare's Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use requests and the url to download all of the sonnets - save the result to `data`
data = requests.get(url_shakespeare_sonnets)


In [6]:
# extract the downloaded text from the requests object and save it to `raw_text_data`
# hint: take a look at the attributes of `data`
# YOUR CODE HERE
raw_text_data = data.text
#raise NotImplementedError()

In [7]:
# check the data type of `raw_text_data`
assert(type(raw_text_data)==str)

### Data Cleaning

In [8]:
# as usual, we need to clean up the messy data
raw_text_data[:3000]

"\ufeffThe Project Gutenberg eBook of Shakespeare's Sonnets\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nRelease date: September 1, 1997 [eBook #1041]\r\n                Most recently updated: March 10, 2024\r\n\r\nLanguage: English\r\n\r\nCredits: the Project Gutenberg Shakespeare Team\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\nI\r\n\r\nFrom fairest creatures we desire increase,\r\nThat there

In [9]:
# Which characters could we use with the split() method to split the text into lines?

# split the text into **lines** and save the result to `split_data`

# YOUR CODE HERE
split_data =raw_text_data.split('\n')
# raise NotImplementedError()

In [10]:
# we need to drop all the boiler plate text (i.e. titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 
split_data[:80] 

["\ufeffThe Project Gutenberg eBook of Shakespeare's Sonnets\r",
 '    \r',
 'This ebook is for the use of anyone anywhere in the United States and\r',
 'most other parts of the world at no cost and with almost no restrictions\r',
 'whatsoever. You may copy it, give it away or re-use it under the terms\r',
 'of the Project Gutenberg License included with this ebook or online\r',
 'at www.gutenberg.org. If you are not located in the United States,\r',
 'you will have to check the laws of the country where you are located\r',
 'before using this eBook.\r',
 '\r',
 "Title: Shakespeare's Sonnets\r",
 '\r',
 'Author: William Shakespeare\r',
 '\r',
 'Release date: September 1, 1997 [eBook #1041]\r',
 '                Most recently updated: March 10, 2024\r',
 '\r',
 'Language: English\r',
 '\r',
 'Credits: the Project Gutenberg Shakespeare Team\r',
 '\r',
 '\r',
 "*** START OF THE PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r",
 'THE SONNETS\r',
 '\r',
 'by William Shakespeare\r',
 '\r

**Use list index slicing to remove the titles and descriptions, so we only have the sonnets.**


In [12]:
# we need to drop all the boilerplate text (i.e., titles and descriptions) as well as extra white spaces
# so that we are left with only the sonnets themselves 

# find index boundaries (start, end)so that 
# sonnets exist between these indices 
# titles and descriptions exist outside of these indices

# use index slicing to isolate the sonnet lines from the text - save the result to `sonnets`

# YOUR CODE HERE
# raise NotImplementedError()
start_index = None
end_index = None

for i, line in enumerate(split_data):
    if '*** START OF THE PROJECT GUTENBERG EBOOK SHAKESPEARE\'S SONNETS ***' in line:
        start_index = i + 1
    if '*** END OF THE PROJECT GUTENBERG EBOOK SHAKESPEARE\'S SONNETS ***' in line:
        end_index = i
        break


if start_index is not None and end_index is not None:

    sonnets = split_data[start_index:end_index]
else:
    print("start or end marker not found.")

# sonnets_text = '\n'.join(sonnets)
# print(sonnets_text[:1000])

Notice that there are many lines that should not be counted as sonnets!

In [13]:
# these non-sonnet lines have far fewer characters than the actual sonnet lines?

sonnets[200:240]

['And sable curls, all silvered o’er with white;\r',
 'When lofty trees I see barren of leaves,\r',
 'Which erst from heat did canopy the herd,\r',
 'And summer’s green all girded up in sheaves,\r',
 'Borne on the bier with white and bristly beard,\r',
 'Then of thy beauty do I question make,\r',
 'That thou among the wastes of time must go,\r',
 'Since sweets and beauties do themselves forsake\r',
 'And die as fast as they see others grow;\r',
 '    And nothing ’gainst Time’s scythe can make defence\r',
 '    Save breed, to brave him when he takes thee hence.\r',
 '\r',
 'XIII\r',
 '\r',
 'O! that you were your self; but, love you are\r',
 'No longer yours, than you yourself here live:\r',
 'Against this coming end you should prepare,\r',
 'And your sweet semblance to some other give:\r',
 'So should that beauty which you hold in lease\r',
 'Find no determination; then you were\r',
 'Yourself again, after yourself’s decease,\r',
 'When your sweet issue your sweet form should bear.\r',

In [14]:
# use your judgement to decide on a good value for  
#   the  minimum number of characters that a sonnet should have
#   call it n_chars
n_chars = 40 

# Let's use that observation to filter out all the non-sonnet lines!
#    save results to `filtered_sonnets`
# Hint: use a list comprehension
filtered_sonnets = [line for line in sonnets if len(line) > n_chars]

# YOUR CODE HERE
# raise NotImplementedError()
print(filtered_sonnets[:20])

['From fairest creatures we desire increase,\r', 'That thereby beauty’s rose might never die,\r', 'But as the riper should by time decease,\r', 'But thou, contracted to thine own bright eyes,\r', 'Feed’st thy light’s flame with self-substantial fuel,\r', 'Thyself thy foe, to thy sweet self too cruel:\r', 'Thou that art now the world’s fresh ornament,\r', 'Within thine own bud buriest thy content,\r', 'And tender churl mak’st waste in niggarding:\r', '    Pity the world, or else this glutton be,\r', '    To eat the world’s due, by the grave and thee.\r', 'When forty winters shall besiege thy brow,\r', 'And dig deep trenches in thy beauty’s field,\r', 'Thy youth’s proud livery so gazed on now,\r', 'Will be a tatter’d weed of small worth held:\r', 'Then being asked, where all thy beauty lies,\r', 'Where all the treasure of thy lusty days;\r', 'To say, within thine own deep sunken eyes,\r', 'Were an all-eating shame, and thriftless praise.\r', 'How much more praise deserv’d thy beauty’s us

In [16]:
# ok - much better!
# but we still need to remove all the punctuation and case normalize the text
filtered_sonnets

['From fairest creatures we desire increase,\r',
 'That thereby beauty’s rose might never die,\r',
 'But as the riper should by time decease,\r',
 'But thou, contracted to thine own bright eyes,\r',
 'Feed’st thy light’s flame with self-substantial fuel,\r',
 'Thyself thy foe, to thy sweet self too cruel:\r',
 'Thou that art now the world’s fresh ornament,\r',
 'Within thine own bud buriest thy content,\r',
 'And tender churl mak’st waste in niggarding:\r',
 '    Pity the world, or else this glutton be,\r',
 '    To eat the world’s due, by the grave and thee.\r',
 'When forty winters shall besiege thy brow,\r',
 'And dig deep trenches in thy beauty’s field,\r',
 'Thy youth’s proud livery so gazed on now,\r',
 'Will be a tatter’d weed of small worth held:\r',
 'Then being asked, where all thy beauty lies,\r',
 'Where all the treasure of thy lusty days;\r',
 'To say, within thine own deep sunken eyes,\r',
 'Were an all-eating shame, and thriftless praise.\r',
 'How much more praise deser

### Use Custom Data Cleaning Tool 

Use one of the methods in the `data_cleaning_toolkit` to clean your data.

There is an example of this in the guided project.

In [20]:
dctk = data_cleaning_toolkit()
print(dir(dctk))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'char_int', 'clean_data', 'create_X_and_Y', 'create_char_sequences', 'int_char', 'maxlen', 'n_features', 'next_char', 'sequences', 'unique_chars']


In [21]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`
# YOUR CODE HERE
# raise NotImplementedError()
dctk = data_cleaning_toolkit()



['from fairest creatures we desire increase\r', 'that thereby beautys rose might never die\r', 'but as the riper should by time decease\r', 'but thou contracted to thine own bright eyes\r', 'feedst thy lights flame with selfsubstantial fuel\r', 'thyself thy foe to thy sweet self too cruel\r', 'thou that art now the worlds fresh ornament\r', 'within thine own bud buriest thy content\r', 'and tender churl makst waste in niggarding\r', 'pity the world or else this glutton be\r', 'to eat the worlds due by the grave and thee\r', 'when forty winters shall besiege thy brow\r', 'and dig deep trenches in thy beautys field\r', 'thy youths proud livery so gazed on now\r', 'will be a tatterd weed of small worth held\r', 'then being asked where all thy beauty lies\r', 'where all the treasure of thy lusty days\r', 'to say within thine own deep sunken eyes\r', 'were an alleating shame and thriftless praise\r', 'how much more praise deservd thy beautys use\r']


In [23]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# YOUR CODE HERE
# raise NotImplementedError()
def clean_text(text):
    return dctk.clean_data(text)

clean_sonnets = [clean_text(line) for line in filtered_sonnets]
# print(clean_sonnets[:20])

In [24]:
# much better!
display(clean_sonnets)
print(len(clean_sonnets))

['from fairest creatures we desire increase\r',
 'that thereby beautys rose might never die\r',
 'but as the riper should by time decease\r',
 'but thou contracted to thine own bright eyes\r',
 'feedst thy lights flame with selfsubstantial fuel\r',
 'thyself thy foe to thy sweet self too cruel\r',
 'thou that art now the worlds fresh ornament\r',
 'within thine own bud buriest thy content\r',
 'and tender churl makst waste in niggarding\r',
 'pity the world or else this glutton be\r',
 'to eat the worlds due by the grave and thee\r',
 'when forty winters shall besiege thy brow\r',
 'and dig deep trenches in thy beautys field\r',
 'thy youths proud livery so gazed on now\r',
 'will be a tatterd weed of small worth held\r',
 'then being asked where all thy beauty lies\r',
 'where all the treasure of thy lusty days\r',
 'to say within thine own deep sunken eyes\r',
 'were an alleating shame and thriftless praise\r',
 'how much more praise deservd thy beautys use\r',
 'if thou couldst answ

1686


### Use Your Data Tool to Create Character Sequences 
for the LSTM model

We'll need the `create_char_sequences` method for this task. <br>
However, this method requires a parameter called `maxlen,` which is responsible for setting the maximum sequence length. 

So what would be a good sequence length, exactly? 

To answer that question, let's do some statistics! 

In [31]:
def calc_stats(corpus):
    """
    Calculates statistics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnet's line length - save the results to `doc_lens` 
    doc_lens = [len(line) for line in corpus]
    # use numpy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code
    # YOUR CODE HERE
    mean = np.mean(doc_lens)
    median = np.median(doc_lens)
    std = np.std(doc_lens)
    max_ = np.max(doc_lens)
    min_ = np.min(doc_lens)
    
    return mean, median, std, max_, min_



In [32]:
# sonnet line length statistics 
mean, med, std, max_, min_ = calc_stats(clean_sonnets)
mean, med, std, max_, min_

(43.30189798339264, 43.0, 3.238936266960386, 58, 32)

In [34]:
# from the results of the sonnet line length statistics, use your judgement to select a value for maxlen
#   hint -- a good value might be half the median length of a sonnet line
# use .create_char_sequences() to create sequences

# YOUR CODE HERE
# raise NotImplementedError()

maxlen = 21

dctk.create_char_sequences(clean_sonnets, maxlen=maxlen)

Created 14935 sequences.


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first 4 lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. <br>
Let's call them in the cells below.

In [35]:
# number of input features for our LSTM model
dctk.n_features

28

In [36]:
# unique characters that appear in our sonnets 
dctk.unique_chars

['e',
 ' ',
 'u',
 'c',
 'q',
 'g',
 'f',
 't',
 'b',
 'v',
 'y',
 'h',
 'j',
 's',
 'k',
 'w',
 'l',
 'o',
 'n',
 'i',
 'x',
 'z',
 '\r',
 'p',
 'd',
 'm',
 'a',
 'r']

## Time for Questions 

----
**Question 1:** 

Why are the `number of unique characters` (i.e., **dctk.unique_chars**) and the `number of model input features` (i.e., **dctk.n_features**) the same?

**Hint:** The model that we will shortly build here is very similar to the text generation model we built in the guided project.

**Answer 1:**

Write your answer here:
In a character-level text generation model, each unique character in the text is treated as a distinct feature for the model to learn from. Therefore, the number of unique characters represents the total set of possible characters the model can predict, and this directly corresponds to the number of model input features. Each character becomes a feature in a one-hot encoding scheme, where the input features for the model are represented as binary vectors, each indicating the presence or absence of a character.


**Question 2:**

Take a look at the printout of `dctk.unique_chars` one more time. Notice that there is a white space. 

Why is it desirable to have a white space as a possible character to predict?

**Answer 2:**

Write your answer here: Including white space as a possible character for prediction is crucial for text generation because:

Preserving Structure: White spaces are essential for maintaining the structure and readability of the text. Without them, the generated text could become a continuous string of characters, which would be difficult to read and interpret.

Word Separation: White spaces serve as delimiters between words. By predicting white spaces, the model can generate coherent and meaningful text with properly separated words.

Natural Language Representation: In natural language, white spaces are integral to representing and generating text in a human-readable format. Including them in the character set allows the model to better mimic natural text patterns and structure.

----

### Use Our Data Tool to Create X and Y Splits

You'll need the `create_X_and_Y` method for this task. 

In [39]:
# TODO: provide a walkthrough of data_cleaning_toolkit with unit tests that check for understanding 
X, y = dctk.create_X_and_Y()
def test_create_X_and_Y():
    dctk = data_cleaning_toolkit()
    dctk.create_char_sequences(['hello world', 'test sequence'], maxlen=5, step=1)
    x, y = dctk.create_X_and_Y()
    assert x.shape[1] == 5  # Should match maxlen
    assert x.shape[2] == len(dctk.unique_chars)  # Should match number of unique characters
    assert y.shape[1] == len(dctk.unique_chars)  # Should match number of unique characters
test_create_X_and_Y()


Created 20 sequences.


![](https://miro.medium.com/max/891/0*jGB1CGQ9HdeUwlgB)

In [40]:
# notice that our input array isn't a matrix - it's a rank three tensor
X.shape

(14935, 21, 28)

In $X$.shape, we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [41]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
         True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,
        False, False, False, False, False, False, False, False, False,
        False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False,  True, False,
        False],
       [False,  True, False, False, False, False, False, False, False,
        False

Notice that each sequence (i.e., $X[i]$ where $i$ is some index value) is `maxlen` long and <br>
has a number of features equal to `dctk.n_features`. <br>Let's try to understand this shape.

In [42]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(21, 28)

**Each row corresponds to a character vector,** and there is `maxlen` number of character vectors. 

**Each column corresponds to a unique character,** and there are `dctk.n_features` number of features. 


In [43]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

Notice that there is a single `True` value, and all the rest of the values are `False`. 

This is a one-hot encoding for which character appears at each index within a sequence. Specifically, the cell above is looking at the first character in the sequence.

Only a single character can appear as the first character in a sequence, so there will be a single `True` value, and the rest will be `False`. 

Let's say that `True` appears in the $ith$ index; by  $ith$ index we mean some index in the general case. So how can we find out which character corresponds to?

To answer this question, we need to use the character-to-integer look-up dictionary. 

In [44]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that character vector is endcoding for
dctk.int_char

{0: 'e',
 1: ' ',
 2: 'u',
 3: 'c',
 4: 'q',
 5: 'g',
 6: 'f',
 7: 't',
 8: 'b',
 9: 'v',
 10: 'y',
 11: 'h',
 12: 'j',
 13: 's',
 14: 'k',
 15: 'w',
 16: 'l',
 17: 'o',
 18: 'n',
 19: 'i',
 20: 'x',
 21: 'z',
 22: '\r',
 23: 'p',
 24: 'd',
 25: 'm',
 26: 'a',
 27: 'r'}

In [46]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
Sequence length: 21


## Time for Questions 

----
**Question 1:** 

In your own words, how would you describe the numbers from the shape printout of `X.shape` to a classmate?


**Answer 1:**

Write your answer here:
To explain the shape of the tensor X (i.e., X.shape), you can break it down into three dimensions, each representing a different aspect of the data:

Number of Samples (n1): This is the first number in the shape tuple. It represents how many sequences (or samples) you have in your dataset. Each sequence corresponds to a specific slice of text data. For example, if X.shape is (14935, 21, 28), then 14935 indicates that there are 14,935 such sequences.

Sequence Length (n2): This is the second number and it represents the length of each sequence. In the case of X.shape being (14935, 21, 28), the 21 indicates that each sequence consists of 21 characters.

Number of Features (n3): The third number represents the number of features or unique characters in the dataset. For X.shape of (14935, 21, 28), the 28 means that there are 28 unique characters that can appear in each position of the sequence.

----


### Build a Shakespeare Sonnet Text Generation Model

Now that we have prepped our data (and understood that process), let's finally build out our character generation model, similar to what we did in the guided project.<br>

First, we'll create a callback to monitor the training -- by printing a sample of text generated by the model at the end of each epoch.

Helper function to generate a sample character:

In [49]:
def sample(preds, temperature=1.0):
    """
    Helper function to generate a sample character
    Input is a predictions vector from our model, for example a set of 27 character probabilities
    Output is the index of the generated character 
    """
    # convert predictions to an array 
    preds = np.asarray(preds).astype('float64')

    # use the temperature hyper-parameter to "warp" (sharpen or spread out) the probability distribution 
    preds = np.log(preds) / temperature

    # use the softmax activation function to create a new list of probabilities 
    #   corresponding to the "warped" probability distribution
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    # Draw a single sample from a multinomial distribution, given these probabilities
    #   The sample will be a one-hot encoded character
    """ Notes on the np.random.multinomial() function 
       The first argument is the number of "trials" we want: 1 in this case
       The second argument is the list of probabilities for each character
       The third argument is number of sets of "trials" we want: again, 1 in this case
       By analogy with a dice-rolling experiment: 
          This "trial" consists of generating a single "throw" of a die with 27 faces;
             each face corresponds to a character and its associated probability
    """

    probas = np.random.multinomial(1, preds, 1)
    
    # return the index that corresponds to the max probability 
    return np.argmax(probas)
text = " ".join(clean_sonnets)


Create the `on_epoch_end` function to be passed into `LambdaCallback()`

In [50]:
def on_epoch_end(epoch, _):
    """"
    Function invoked at the end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunece into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next maxlen chars should be that follow the seed string
    for i in range(maxlen):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [51]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)
print(f'All of Shakespeare\'s sonnets comprise about {len(text)} characters')

All of Shakespeare's sonnets comprise about 74692 characters


Create the callback object

In [54]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

----
### Build and Train Model

Build a text generation model using LSTMs. Feel free to reference the model used in the guided project. 

It is recommended that you train this model to at least 50 epochs (but more if you're computer can handle it). 

You are free to change up the architecture as you wish. 

Just in case you have difficultly training a model, there is a pre-trained model saved to a file called `trained_text_gen_model.h5` that you can load (in the same way that you learned how to load in Keras models in Sprint 2 Module 4). 

In [58]:
# build text generation model layer by layer 
# fit model

# YOUR CODE HERE
# raise NotImplementedError()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(dctk.maxlen, dctk.n_features), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(128, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(dctk.n_features))
model.add(tf.keras.layers.Activation('softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01))
model.fit(X, y, batch_size=128, epochs=50, callbacks=[print_callback])

Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 2.9936
----- Generating text after Epoch: 0
 be"- Generating with seed: "ature bankrupt is
 bere bankrupt is ny whe sim the uakit
117/117 ━━━━━━━━━━━━━━━━━━━━ 15s 90ms/step - loss: 2.9914
Epoch 2/50
116/117 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 2.2891
----- Generating text after Epoch: 1
 all men m"ating with seed: "eetest bud
 all men my hanetorenthat the
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 87ms/step - loss: 2.2879
Epoch 3/50
116/117 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 2.0496
----- Generating text after Epoch: 2
----- Generating with seed: "hy odour matcheth not"
hy odour matcheth not by ned nadd lpokes f
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - loss: 2.0492
Epoch 4/50
116/117 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 1.9014
----- Generating text after Epoch: 3
 but being both "with seed: "iend
 but being both alt ho though thomy s
117/117 ━━━━━━━━━━━━━━━━━━━━ 10s 83ms/step - loss: 1.9013
Epoch 5/50
116/117 ━━━━━━

### Save the trained model to a file

In [59]:
# save trained model to file 
model.save("trained_text_gen_model.h5")

### Let's Play With Our Trained Model 

Now that we have a trained model that, though far from perfect, can generate actual English words, we can look at the predictions to continue learning more about how a text generation model works.

We can also take this as an opportunity to unpack the `def on_epoch_end` function to understand better how it works. 

In [60]:
# this is our joined clean sonnet data
text

'from fairest creatures we desire increase\r that thereby beautys rose might never die\r but as the riper should by time decease\r but thou contracted to thine own bright eyes\r feedst thy lights flame with selfsubstantial fuel\r thyself thy foe to thy sweet self too cruel\r thou that art now the worlds fresh ornament\r within thine own bud buriest thy content\r and tender churl makst waste in niggarding\r pity the world or else this glutton be\r to eat the worlds due by the grave and thee\r when forty winters shall besiege thy brow\r and dig deep trenches in thy beautys field\r thy youths proud livery so gazed on now\r will be a tatterd weed of small worth held\r then being asked where all thy beauty lies\r where all the treasure of thy lusty days\r to say within thine own deep sunken eyes\r were an alleating shame and thriftless praise\r how much more praise deservd thy beautys use\r if thou couldst answer this fair child of mine\r shall sum my count and make my old excuse\r this wer

In [61]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

15249

In [62]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e., input sequence into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

In [63]:
# display the "seed string" i.e. the input sequence into the model
print('----- Input seed: "' + sentence + '"')

----- Input seed: "ng with the trophies "


In [64]:
# use model to predict what the next maxlen chars should be that follow the seed string
for i in range(maxlen):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
    # i.e. create a numerical encoding for each char in the sequence 
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [65]:
# this is the seed string
generated

'ng with the trophies '

In [66]:
# these are the maxlen chars that the model thinks should come after the seed stirng
sentence

'whee gre it not those'

In [67]:
# how put it all together
generated + sentence

'ng with the trophies whee gre it not those'

# Resources and Stretch Goals

## Stretch Goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g., plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_generation) - code for training an RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem and provides an example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN